## Data

Guess there are a few so let's see if I can find these on NCBI 
looked https://www.ncbi.nlm.nih.gov/assembly/?term=klebsiella+pneumoniae%5Borgn%5D++20_GR_12
and searcher for the name plus the strain

From download, selected RefSeq and downloaded the gff, genome, and CDS
https://www.ncbi.nlm.nih.gov/genome/?term=Klebsiella+pneumoniae
Went to the reference genome, and selected the Chr then clicked on assembly and downloaded that for each

|Name|link|
|:----|:----|
|Klebsiellapneumoniae-20_GR_12| https://www.ncbi.nlm.nih.gov/assembly/GCF_001701845.2|
|Klebsiellapneumoniae-1_GR_13|  https://www.ncbi.nlm.nih.gov/assembly/GCF_001701425.2|
|Klebsiellapneumoniae-16_GR_13|    https://www.ncbi.nlm.nih.gov/assembly/GCF_001701835.2|
|Klebsiellapneumoniae-2_GR_12| https://www.ncbi.nlm.nih.gov/assembly/GCF_001702045.2|
|Xylellafastidiosa-stags_leap| https://www.ncbi.nlm.nih.gov/assembly/GCF_021459925.1|
|Streptococcusmutans-B04Sm5| https://www.ncbi.nlm.nih.gov/assembly/GCF_014621675.1|  No results were found for the others|
|Vibrioparahaemolyticus-RIMD_2210633| https://www.ncbi.nlm.nih.gov/assembly/GCF_000196095.1/|
|Listeriamonocytogenes-ref| https://www.ncbi.nlm.nih.gov/assembly/GCF_000196035.1 |used the reference since the strains didn't seem specific|
|Staphylococcusaureus-USA300| https://www.ncbi.nlm.nih.gov/assembly/GCF_022226995.1    | there were three| so I selected complete genomePandoraeafibrosis-6399| https://www.ncbi.nlm.nih.gov/assembly/GCF_000807775.2Pandoraeafibrosis-7641| https://www.ncbi.nlm.nih.gov/assembly/GCF_000807785.2|
|Escherichiacoli-K_12|   https://www.ncbi.nlm.nih.gov/assembly/GCF_000005845.2| downloaded the reference for escherichia coli k-12[orgn]Escherichiacoli-E2348_69|    https://www.ncbi.nlm.nih.gov/assembly/GCF_000026545.1|
|Pseudomonasaeruginosa-UCBPP_PA14| https://www.ncbi.nlm.nih.gov/assembly/GCF_000014625.1|


Data are now in SRA format, so we need to convert them back to the fastq format
https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit
https://github.com/ncbi/sra-tools
downloaded version 3.0 sratoolkit.3.0.0-ubuntu64.tar.gz for linux and unziped (`tar -xvzf  sratoolkit.3.0.0-ubuntu64.tar.gz`)

Added the sra toolkit bin to my path:

`export PATH=" full path to sratoolkit.3.0.0-ubuntu64/bin:$PATH"`

Then you can run: `fastq-dump --gzip {filename}` for each of the accessions.


Hmmm looking at the data they have 't' rather than 'U' so look to see if the non-coding versions do too...
https://www.ebi.ac.uk/ena/browser/view/SRR14498249


In [ ]:
## Have a look in each gff at which genes are shared between the species/strains that we have
genome_dir = '' # wherever teh genomes are

query_list = [g for g in os.listdir(genome_dir) if 'gff' in g and 'genomic' in g]
from sciutil import SciUtil

u = SciUtil()
for gff_file in query_list:
    rows = []
    f_name = gff_file.replace('_genomic.gff', '')
    u.dp([f_name])
    with open(f'{genome_dir}{gff_file}', 'r') as gff:
        for line in gff:
            if line[0] == '#':
                continue
            else:
                line = line.split('\t')
                line[-1] = line[-1].strip()
                rows.append([f'{line[0]}:{line[3]}-{line[4]}'] + line)
    gff_df = pd.DataFrame(rows)
    # This is indeed the case!!!! OMG how annoying cmon
    print(len(df), len(set(gff_df[0].values)))

    df = gff_df[gff_df[2] == 'RefSeq']
    print(len(df), len(set(df[0].values))) # Still got dups
    print(df[3].value_counts())
    # OK so let's just keep the genes and the tRNA, rRNA, and ncRNAs
    df = df[df[3].isin(['gene', 'tRNA', 'ncRNA', 'rRNA'])]
    print(len(df), len(set(df[0].values))) # Still got dups
    # Sort by '3' and drop duplicates
    df = df.sort_values(by=[0, 3])
    #print(df[df.duplicated([0], keep=False)].sort_values(by=[0]).head()) # Can see that it is mainly the genes/trnas
    df = df.drop_duplicates(subset=[0], keep='last')
    print(len(df), len(set(df[0].values))) # Still got dups
    #print(df[3].value_counts()) # Can see that it is mainly the genes/trnas
    # Save to tsv without the header and also drop that first column (i.e. save rows 1 - 9)
    df = df.drop(columns=[0])
    df.to_csv(f'{genome_dir}{gff_file.replace("genomic", "genes-RNA")}', sep='\t', index=False, header=False)

## Some useful functions

In [1]:
def gff_to_bed(bedfile, gff_file, filter_value='') -> None:
    """
    Since all tools have a stupid amount of deps just write a simple parser for a gff file
    NC_003210.1 RefSeq  region  1   2944528 .   +   .   ID=NC_003210.1:1..2944528;Dbxref=taxon:169963;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=EGD-e
    """
    # BED output
    # transcript:YNL036W_mRNA 0   499 cd2b3759-e172-437d-a480-f0a0eafecf75    0   +   666 998
    # chromosome, start, end, name, score, strand, thickStart, thickEnd, itemRgb, blockCount, blockSizes, blockStarts
    with open(bedfile, 'w+') as bed:
        with open(gff_file, 'r') as gff:
            for line in gff:
                if line[0] == '#':
                    continue
                else:
                    line = line.split('\t')
                    if filter_value is not None:
                        # Perhaps subtract 1 frmo the start if you are doing bacteria
                        if filter_value == line[2]:  # i.e. make sure that the line contains something of interest e.g. gene
                            bed_line = [line[0], line[3], line[4], line[-1].strip(), line[5], line[6]]
                            bed.write('\t'.join(bed_line) + '\n')
                    else:
                        # Perhaps subtract 1 frmo the start if you are doing bacteria
                        bed_line = [line[0], line[3], line[4], line[-1].strip(), line[5], line[6]]
                        bed.write('\t'.join(bed_line) + '\n')

## Now you can use this to map to your transcriptome

In [ ]:
import os

genome_fasta = 'path to fasta'
output_reference_name = 'path to output transcriptome'
gff_file = ' output to file you created before'

# This has calls around some of these mappings (bed works for gff or bed files)
os.system(' '.join(['bedtools', 'getfasta', '-fi', genome_fasta, '-bed', gff_file, '-fo', output_reference_name]))

## Converting fastq to bam files


In [ ]:
 def fastq_to_bam(self, fastq, transcriptome, filename, remove_files=True) -> None:
        """
        Convert the fasta sequence to a sam file, then to a bam, finally sort this.
        We need this for later steps in the pipeline.
        """
        ref = transcriptome  # transcriptome
        fasta = transcriptome + '.fai' # Get the index (note this is expected to have already have been done)
        sam = f'{filename}.sam'
        bam = f'{filename}.bam'
        sorted_bam = f'{filename}.sorted.bam'
        # Run minimap2 --> you'll need to check params for illumina
        os.system(" ".join(['minimap2', '-ax', 'splice', '-uf', '-k14 ', '-t', '20', ref, fastq, '>', sam]))
        # Run bam
        os.system(" ".join(['samtools', 'view', '-S', sam, '-bh', '-t', fasta, '>', bam]))
        # Sort bam
        os.system(" ".join(['samtools', 'sort', bam, '>', sorted_bam]))
        # Index bam
        os.system(" ".join(['samtools', 'index', sorted_bam]))
        # Remove other files
        if remove_files:
            os.system(" ".join(['rm', bam])) # Just leave the sorted bam
            os.system(" ".join(['rm', sam]))

In [ ]:
# Should be goood :) 